In [1]:
library(RPostgreSQL)
source("../aux/keys.R")

library(syuzhet)
library(tm)
library(RColorBrewer)
library(wordcloud)
library(twitteR)
library(topicmodels)
library(ggplot2)
library(dplyr)
library(reshape2)
library(scales)

Loading required package: DBI

Loading required package: NLP


Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate



Attaching package: ‘dplyr’


The following objects are masked from ‘package:twitteR’:

    id, location


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘scales’


The following object is masked from ‘package:syuzhet’:

    rescale




In [2]:
# PostgreSQL
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = db_name,
                 host = db_server, port = db_port,
                 user = db_user, password = db_password)

(total - users - bots) fr and pt
tweets and retweets

Date: '2020-03-20' and '2020-04-09' 
222 total pt
223 total fr

In [5]:
# tweets pt
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE am.codamostra = 222")
tweets_pt <- dbGetQuery(con, cmd)
saveRDS(tweets_pt, "data/tweets_pt.rds")

In [ ]:
# tweets users pt
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE am.codamostra = 224")
tweets_pt <- dbGetQuery(con, cmd)
saveRDS(tweets_pt, "data/tweets_users_pt.rds")

In [ ]:
# tweets bots pt
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE am.codamostra = 226")
tweets_pt <- dbGetQuery(con, cmd)
saveRDS(tweets_pt, "data/tweets_bots_pt.rds")

In [6]:
# tweets fr
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE am.codamostra = 223")
tweets_fr <- dbGetQuery(con, cmd)
saveRDS(tweets_fr, "data/tweets_fr.rds")

In [ ]:
# tweets users fr
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE am.codamostra = 225")
tweets_pt <- dbGetQuery(con, cmd)
saveRDS(tweets_pt, "data/tweets_users_fr.rds")

In [ ]:
# tweets bots pt
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE am.codamostra = 227")
tweets_pt <- dbGetQuery(con, cmd)
saveRDS(tweets_pt, "data/tweets_bots_fr.rds")

In [19]:
# retweets pt
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE lower(t.text) like 'rt%' AND am.codamostra = 222")
retweets_pt <- dbGetQuery(con, cmd)
saveRDS(retweets_pt, "data/retweets_pt.rds")

In [20]:
# retweets fr
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE lower(t.text) like 'rt%' AND am.codamostra = 223")
retweets_fr <- dbGetQuery(con, cmd)
saveRDS(retweets_fr, "data/retweets_fr.rds")

In [21]:
# only tweets pt
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE lower(t.text) not like 'rt%' AND am.codamostra = 222")
only_tweets_pt <- dbGetQuery(con, cmd)
saveRDS(only_tweets_pt, "data/only_tweets_pt.rds")

In [22]:
# only tweets fr
cmd = paste("select 
    datatweet || ' ' || horatweet || ' UTC' as created, 
    text as tweet
    FROM tweet t 
    INNER JOIN amostratweet am ON am.codtweet = t.codtweet 
    WHERE lower(t.text) not like 'rt%' AND am.codamostra = 223")
only_tweets_fr <- dbGetQuery(con, cmd)
saveRDS(only_tweets_fr, "data/only_tweets_fr.rds")

In [7]:
# portuguese tweets
tweets_pt <- readRDS("data/tweets_pt.rds")
tweets_users_pt <- readRDS("data/tweets_users_pt.rds")
tweets_bots_pt <- readRDS("data/tweets_bots_pt.rds")
# french tweets
tweets_fr <- readRDS("data/tweets_fr.rds")
tweets_users_fr <- readRDS("data/tweets_users_fr.rds")
tweets_bots_fr <- readRDS("data/tweets_bots_fr.rds")

## Sentiments

In [9]:
sentimentos <- function (tweets, lang) {
    sentiments <- get_nrc_sentiment(tweets , language=lang)
    tweets <- cbind(tweets, sentiments)
    sentimentTotals <- data.frame(colSums(tweets[,c(2:9)]))
    names(sentimentTotals) <- "count"
    sentimentTotals <- cbind("sentiment" = rownames(sentimentTotals), sentimentTotals)
    rownames(sentimentTotals) <- NULL    
    print(ggplot(data = sentimentTotals, aes(x = sentiment, y = count)) +
      geom_bar(aes(fill = sentiment), stat = "identity") +
      theme(legend.position = "none") +
      xlab("Sentiment") + ylab("Total Count") + ggtitle("Total Sentiment Score for All Tweets"))
    return(sentiments)
}

In [10]:
sent_pt = sentimentos(tweets_pt$tweet, "portuguese")

Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


In [ ]:
sent_users_pt = sentimentos(tweets_users_pt$tweet, "portuguese")

In [ ]:
sent_bots_pt = sentimentos(tweets_bots_pt$tweet, "portuguese")

In [ ]:
sent_fr = sentimentos(tweets_fr$tweet, "french")

In [ ]:
sent_users_fr = sentimentos(tweets_users_fr$tweet, "french")

In [ ]:
sent_bots_fr = sentimentos(tweets_bots_fr$tweet, "french")

## Polaridade

## Termos Frequentes